## Predicting with a DLWP-CS model

Finally we will explore using the advanced functionality in DLWP-CS to make a time-series global weather prediction with our trained DLWP-CS model. We will save the prediction to a netCDF file and apply inverse scaling to get physical variables back. Again, I recommend having this model run on a GPU with at least 4 GB of video memory.

#### Required packages

No new packages are needed here beyond the main DLWP-CS requirements in the README.

### Parameters

Some user-specified parameters. The `scale_file` contains the mean and standard of the data (which was dropped in the cubed sphere remapping). The `map_files` were produced by the cubed sphere remapping. We can re-use them here so we don't have to generate them again.

In [6]:
import os
os.chdir(os.pardir)

root_directory = '/usr/local/google/ilopezgp/ERA5_data_dlwp'
predictor_file = os.path.join(root_directory, 'tutorial_z500_CS.nc')
scale_file = os.path.join(root_directory, 'tutorial_z500.nc')

model = os.path.join(root_directory, 'dlwp-cs_tutorial')
map_files = ('./map_LL91x180_CS48.nc', './map_CS48_LL91x180.nc')

We'll resurrect some parameters from the training tutorial. See that notebook for definitions. Note that we omit `data_interval` because we simply select only every 6 hours from the dataset.

In [7]:
io_selection = {'varlev': ['z/500']}
add_solar = True
io_time_steps = 2

Specify the data used for prediction. We'll make weekly forecasts in the test set, initialized at 0 UTC. We need to specify a subset of the data that contains all these initializations.

In [8]:
import numpy as np
import pandas as pd
import xarray as xr

validation_set = pd.date_range('2016-12-31', '2018-12-31', freq='6H')
validation_set = np.array(validation_set, dtype='datetime64[ns]')

Set the initialization dates, the numer of foreward forecast hours, and the time step (could be automated...)

In [9]:
dates = pd.date_range('2017-01-01', '2018-12-31', freq='7D')
initialization_dates = xr.DataArray(dates)
num_forecast_hours = 5 * 24
dt = 6

### Load the DLWP model

In [10]:
from DLWP.util import load_model, remove_chars, is_channels_last

dlwp = load_model(model)

# File to save the forecast
forecast_file = os.path.join(root_directory, 'forecast_%s.nc' % remove_chars(model.split(os.sep)[-1]))

/usr/local/google/home/ilopezgp/DLWP-CS/DLWP/model/models_torch.py:25: UserWarning: DLWPTorchNN is not available because PyTorch is not installed.
  warnings.warn('DLWPTorchNN is not available because PyTorch is not installed.')


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

### Open the data and create the data generator

In [21]:
all_ds = xr.open_dataset(predictor_file)
predictor_ds = all_ds.sel(sample=validation_set)

In [22]:
from DLWP.model import SeriesDataGenerator

sequence = dlwp._n_steps if hasattr(dlwp, '_n_steps') and dlwp._n_steps > 1 else None
val_generator = SeriesDataGenerator(dlwp, predictor_ds, rank=3, add_insolation=add_solar,
                                    input_sel=io_selection, output_sel=io_selection,
                                    input_time_steps=io_time_steps, output_time_steps=io_time_steps,
                                    shuffle=False, sequence=sequence, batch_size=32,
                                    load=False, channels_last=is_channels_last(dlwp))

### Create the estimator and make a prediction

We use the handy TimeSeriesEstimator class to intelligently produce a time series forecast. This class depends on the model and the data generator.

In [23]:
from DLWP.model import TimeSeriesEstimator

estimator = TimeSeriesEstimator(dlwp, val_generator)

/usr/local/google/home/ilopezgp/DLWP-CS/DLWP/model/extensions.py:40: UserWarning: DLWPFunctional models are only partially supported by TimeSeriesEstimator. The inputs/outputs to the model must be the same if the model predicts a sequence.
  warnings.warn('DLWPFunctional models are only partially supported by TimeSeriesEstimator. The '


In [24]:
print('Predicting with model %s...' % model)

# Select the samples from the initialization dates. The first "time" input to the model is actually one time step earlier
samples = np.array([int(np.where(val_generator.ds['sample'] == s)[0]) for s in initialization_dates]) \
    - io_time_steps + 1
time_series = estimator.predict(num_forecast_hours // dt, samples=samples, verbose=1)

Predicting with model /usr/local/google/ilopezgp/ERA5_data_dlwp/dlwp-cs_tutorial...
Time step 1/5
4/4 [==============================] - 16s 411ms/step
Time step 2/5
4/4 [==============================] - 2s 485ms/step
Time step 3/5
4/4 [==============================] - 2s 464ms/step
Time step 4/5
4/4 [==============================] - 2s 448ms/step
Time step 5/5
4/4 [==============================] - 2s 467ms/step


In [25]:
# Transpose if channels_last was used for the model
if is_channels_last(dlwp):
    time_series = time_series.transpose('f_hour', 'time', 'varlev', 'x0', 'x1', 'x2')

Scale the variables back to real data.

In [26]:
if scale_file is None:
    scale_ds = predictor_ds
else:
    scale_ds = xr.open_dataset(scale_file)
sel_mean = scale_ds['mean'].sel(io_selection)
sel_std = scale_ds['std'].sel(io_selection)
time_series = time_series * sel_std + sel_mean

For some reason the time series output, when saved to netCDF, is not compatible with TempestRemap. I have yet to figure out why. But there is a function in the DLWP `verify` module that re-formats a time series and produces output that TempestRemap is happy with...

In [27]:
from DLWP.verify import add_metadata_to_forecast_cs

fh = np.arange(dt, time_series.shape[0] * dt + 1., dt)
time_series = add_metadata_to_forecast_cs(
    time_series.values,
    fh,
    predictor_ds.sel(**io_selection).sel(sample=initialization_dates)
)

Save the prediction in cubed sphere format. Drop the string "varlev" coordinate for TempestRemap.

In [28]:
time_series.drop('varlev').to_netcdf(forecast_file + '.cs')

### Remap the forecast to a latitude-longitude grid

In [30]:
from DLWP.remap import CubeSphereRemap
csr = CubeSphereRemap(to_netcdf4=True)
csr.assign_maps(*map_files)
csr.convert_from_faces(forecast_file + '.cs', forecast_file + '.tmp')
csr.inverse_remap(forecast_file + '.tmp', forecast_file, '--var', 'forecast')
os.remove(forecast_file + '.tmp')

CubeSphereRemap.convert_from_faces: loading data to memory...
CubeSphereRemap: assigning new coordinates to dataset
CubeSphereRemap.convert_from_faces: exporting data to file /usr/local/google/ilopezgp/ERA5_data_dlwp/forecast_dlwp-cs_tutorial.nc.tmp...
CubeSphereRemap.convert_from_faces: successfully exported reformatted data
CubeSphereRemap: applying inverse map...
/usr/local/google/home/ilopezgp/anaconda3/envs/dlwp2/bin/ApplyOfflineMap --in_data /usr/local/google/ilopezgp/ERA5_data_dlwp/forecast_dlwp-cs_tutorial.nc.tmp --out_data /usr/local/google/ilopezgp/ERA5_data_dlwp/forecast_dlwp-cs_tutorial.nc --map ./map_CS48_LL91x180.nc --var forecast
CubeSphereRemap: successfully inverse remapped data into /usr/local/google/ilopezgp/ERA5_data_dlwp/forecast_dlwp-cs_tutorial.nc


That was fast, right?